# 7506 - Trabajo Práctico 2

---

## Introducción

### Librerías

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import matplotlib.pyplot as plt
from matplotlib import style

In [2]:
# Dataset Train preprocesado
train_ds = pd.read_csv('datasets/tp1-train.csv')
train_ds = train_ds.drop(['Unnamed: 0'], axis=1)
train_ds.head()

,start_date,end_date,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,2021-09-17,2021-11-03 00:00:00.000000000,-34.644207,-58.498511,Mataderos,PH,3.0,2.0,73.0,73.0,135000.0,Excelente Ph 3 amb.- Venta-Barrio Naon
1,2021-11-04,2021-11-12 00:00:00.000000000,-34.617285,-58.396332,Balvanera,Departamento,1.0,1.0,43.0,43.0,59800.0,Departamento - Balvanera
2,2021-06-24,2021-10-28 00:00:00.000000000,-34.587229,-58.420653,Palermo,Departamento,4.0,3.0,118.0,118.0,229000.0,OPORTUNIDAD IMPECABLE SEMIPISO DE 118 m2 EN P...
3,2021-11-23,2021-12-03 00:00:00.000000000,-34.621213,-58.432909,Caballito,Departamento,4.0,3.0,113.0,107.0,320000.0,Departamento - Caballito - Venta - 4 Ambientes...
4,2021-06-22,2022-07-17 00:00:00.000000000,-34.562502,-58.453964,Belgrano,Departamento,1.0,1.0,24.0,24.0,60000.0,Departamento - Belgrano Barrancas


In [3]:
# Dataset Test preprocesado
test_ds = pd.read_csv('datasets/tp1-test.csv')
test_ds = test_ds.drop(['Unnamed: 0'], axis=1)
test_ds

,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title
0,-34.632666,-58.483515,Floresta,PH,3.0,2.0,78.0,72.0,110000.0,PH 3 Ambientes con patio Floresta Sin Expensas
1,-34.592577,-58.398797,Barrio Norte,Departamento,2.0,1.0,32.0,27.0,95000.0,U$S 95.000 - 2 amb - Recoleta - Azcuénaga 1400...
2,-34.626402,-58.456731,Caballito,Departamento,2.0,1.0,40.0,40.0,83000.0,VENTA al frente con balcon - con renta 2AMB
3,-34.607440,-58.421899,Almagro,Departamento,3.0,2.0,81.0,68.0,165000.0,"HERMOSO DEPTO SIN EXP. 3 AMB.RECICLADO, PATIO"
4,-34.585663,-58.415008,Palermo,Departamento,4.0,2.0,75.0,75.0,290000.0,OM BOTÁNICO: Departamento a estrenar en Palermo
...,...,...,...,...,...,...,...,...,...,...
16658,-34.611851,-58.383438,Monserrat,Departamento,3.0,2.0,55.0,45.0,72500.0,Depto 3 Ambientes En Venta En Monserrat Con Patio
16659,-34.562502,-58.453964,Belgrano,Departamento,1.0,1.0,36.0,36.0,130000.0,Departamento - Belgrano
16660,-34.611979,-58.403334,Balvanera,PH,7.0,5.0,364.0,233.0,355000.0,"Balvanera PH , Excelente Unidad de 2 plantas c..."
16661,-34.578861,-58.484033,Villa Urquiza,Departamento,1.0,1.0,38.0,34.0,125000.0,MONOAMBIENTE | BAUNESS 2000


## Procesamiento del lenguaje natural

### Ampliación del dataset

Este dataset incluye descripciones de las propiedades del otro dataset. Veremos como podemos extraer información de estas descripciones.

In [4]:
descriptions_dataset = pd.read_csv('datasets/properati_argentina_2021_decrip.csv')

In [5]:
descriptions_dataset

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...
...,...,...
460149,MbQ4rqEzjp9Jfc3Sywb/YQ==,Corredor Responsable: Hector Garaygorta / Cris...
460150,fbuG1ohYC623nxLYPQPBvw==,Corredor Responsable: MARIA DEL ROSARIO NEME -...
460151,Ry5hMwctFXD7rLoUpzduxQ==,En cumplimiento de las leyes provinciales vige...
460152,xH37f+l+VOP8Vj0T3g+r6g==,En cumplimiento de las leyes provinciales vige...


#### Tecnica Minqing Hu y Bing Liu

Una forma de analizar el sentimiento de un de un texto es considerando su sentimiento como la suma de los sentimientos de cada una de las palabras que lo forman.

Para el analisis de sentimiento nos guiamos del analisis realizado en esta pagina: https://www.cienciadedatos.net/documentos/py25-text-mining-python.html

Utilizamos algunas funciones de tokenizacion y limpieza de ahi con alguna sutil modificacion para nuestro caso de uso en particular.

In [6]:
def limpiar_tokenizar(texto):
    '''
    Esta función limpia y tokeniza el texto en palabras individuales.
    El orden en el que se va limpiando el texto no es arbitrario.
    El listado de signos de puntuación se ha obtenido de: print(string.punctuation)
    y re.escape(string.punctuation)
    '''
    
    # Se convierte todo el texto a minúsculas
    nuevo_texto = str(texto).lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\\¡\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    # Tokenización por palabras individuales
    nuevo_texto = nuevo_texto.split(sep = ' ')
    # Eliminación de tokens con una longitud < 2
    nuevo_texto = [token for token in nuevo_texto if len(token) > 1]
    
    return(nuevo_texto)

In [7]:
# se aplica la función de limpieza y tokenización a cada descripcion

descriptions_dataset['texto_tokenizado'] = descriptions_dataset['property_description'].apply(lambda x: limpiar_tokenizar(x))
descriptions_dataset[['texto_tokenizado']].head()

,texto_tokenizado
0,"[corredor, responsable, maria, alejandra, geno..."
1,"[hermoso, ph, tipo, casa, de, ambientes, br, a..."
2,"[excelente, ph, en, núñez, amb, oportunidad, t..."
3,"[unico, ph, de, dormitorios, en, impecable, es..."
4,"[ph, amb, villa, crespo, exclusivo, uso, profe..."


In [8]:
# unnest de la columna texto_tokenizado

descriptions_tidy = descriptions_dataset.explode(column='texto_tokenizado')
descriptions_tidy = descriptions_tidy.drop(columns='property_description')
descriptions_tidy = descriptions_tidy.rename(columns={'texto_tokenizado':'token'})
descriptions_tidy.head(3)

,id,token
0,BZCeiGkJr1WBUi6pKJQwJQ==,corredor
0,BZCeiGkJr1WBUi6pKJQwJQ==,responsable
0,BZCeiGkJr1WBUi6pKJQwJQ==,maria


Notamos que las preposiciones no son relevantes para entender que atributo podria ser mejor para expandir el datast, asi que decidimos agregarlas como stopwords.

Tampoco van a variar mucho el analisis de sentimiento realizado en este trabajo.

In [9]:
## listado de stopwords

preposiciones = ["a", "ante", "bajo", "(cabe)", "con", "contra", "de", "desde", "durante", "en", "entre", "hacia", "hasta", "mediante", "para", "por", "según", "sin", "sobre", "tras", "vía"]

stop_words = []

stop_words += preposiciones

# filtrado para excluir stopwords
descriptions_tidy = descriptions_tidy[~(descriptions_tidy["token"].isin(stop_words))]

Agregamos un lexicon en español de esta pagina: https://github.com/jboscomendoza/lexicos-nrc-afinn

In [10]:
# lexicon sentimientos
lexicon = pd.read_csv('lexico_nrc.csv')
lexicon

,palabra,sentimiento,word
0,ciegamente,negativo,blindly
1,ciegamente,tristeza,blindly
2,a distancia,negativo,aloof
3,a hurtadillas,sorpresa,stealthily
4,a la deriva,anticipación,adrift
...,...,...,...
15302,zumbido,anticipación,buzz
15303,zumbido,miedo,buzz
15304,zumbido,positivo,buzz
15305,zumbido,negativo,drone


In [11]:
def mappear_valores_sentimiento(s):
    # 1 Positivo
    # 0 Neutro
    # -1 Negativo
    sentimiento_numerico = 0
    if str(s) in ['negativo', 'tristeza', 'miedo', 'enfado', 'tristeza', 'asco']:
        sentimiento_numerico = -1
    if str(s) in ['sorpresa', 'positivo', 'confianza', 'alegría']:
        sentimiento_numerico = 1
    if str(s) in ['anticipación']:
        sentimiento_numerico = 0
        
    return sentimiento_numerico
    

In [12]:
lexicon['sentimiento'] = lexicon['sentimiento'].apply(lambda x: mappear_valores_sentimiento(x))
lexicon[['sentimiento']].head()

,sentimiento
0,-1
1,-1
2,-1
3,1
4,0


In [35]:
descriptions_tidy[~descriptions_tidy.token.isin(lexicon.palabra)].head()

,id,token
0,BZCeiGkJr1WBUi6pKJQwJQ==,corredor
0,BZCeiGkJr1WBUi6pKJQwJQ==,maria
0,BZCeiGkJr1WBUi6pKJQwJQ==,alejandra
0,BZCeiGkJr1WBUi6pKJQwJQ==,genoveva
0,BZCeiGkJr1WBUi6pKJQwJQ==,bohn


In [36]:
lexicon[~lexicon.palabra.isin(descriptions_tidy.token)].head()

,palabra,sentimiento,word
0,ciegamente,-1,blindly
1,ciegamente,-1,blindly
2,a distancia,-1,aloof
3,a hurtadillas,1,stealthily
4,a la deriva,0,adrift


In [32]:
# sentimiento promedio de cada descripcion
descripcion_sentimientos = pd.merge(
                            left     = descriptions_tidy,
                            right    = lexicon,
                            left_on  = "token", 
                            right_on = "palabra",
                            how      = "inner"
                      )

descripcion_sentimientos = descripcion_sentimientos.drop(columns = ["palabra", "word"])
descripcion_sentimientos

,id,token,sentimiento
0,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
1,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
2,BZCeiGkJr1WBUi6pKJQwJQ==,responsable,1
3,xYlWDFS4uIVIT3eMeZEs9w==,responsable,1
4,xYlWDFS4uIVIT3eMeZEs9w==,responsable,1
...,...,...,...
17186156,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1
17186157,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1
17186158,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1
17186159,EEU4SLi5FwFwIrDbEdM2lw==,débil,-1


In [34]:
#Necesito que aca aparezca tambein el token, no se en que moemento lo pierdo

In [33]:
# suma de los sentimientos de las palabras
descripcion_sentimientos = descripcion_sentimientos[["id", "token", "sentimiento"]] \
                      .groupby(["id"])\
                      .sum().reset_index()
descripcion_sentimientos.head()

,id,sentimiento
0,+++LrfX9dl/rsnbNTgVaSw==,24
1,+++do60YqD8lHYx2VYGDqQ==,1
2,+++wa5fxPpZiosDy8gUAAw==,2
3,+++z1sn9bNyplgPCYXZlyw==,7
4,++/5cFjr0gDqo7tRQJ+crA==,10


In [19]:
descripcion_sentimientos.sentimiento.max()

273

In [20]:
top10_positivas = descripcion_sentimientos[descripcion_sentimientos['sentimiento'] > 248]

In [21]:
top10_positivas

,id,sentimiento
45781,4SE4j0/K7KZhkrrkAue+kg==,250
62240,6o7JTmJGu63NKKRrjfNhKQ==,249
67145,7TYyxaZGxMGIwWjxFl6UPg==,269
151969,JTLEkkzVf4Q3FDkUkQ7KdQ==,270
252779,XmIyOYHVUVPOeW9CeCqaQA==,259
313800,gNTZsMEMRicWrbSpZt6Crw==,273
353040,ltGVQ+iC/gvez0qA5vcq+Q==,263
420985,vW5ymnsy1JQATsHiyldIRw==,270
421558,vbX/XP0EGuuxp93ncSDCUw==,270
441963,yUR6LZhegkZPE8Ug6IvGtA==,269


In [22]:
descripcion_sentimientos.sentimiento.min()

-71

In [23]:
top10_negativas = descripcion_sentimientos[descripcion_sentimientos['sentimiento'] < -33]

In [24]:
top10_negativas

,id,sentimiento
126395,FsPNBpsU5aZxA08udVoiIA==,-36
147415,IqKM9INZW8TiRUrE/xheag==,-66
159262,KWFIk6SR8JFrfLNsfJcoPg==,-37
163689,L7DlI4si9C2ReSvIrpvYcQ==,-41
206906,RHcIDs6WjjS4uBrkGeU5cg==,-36
277546,bGAQx+XUF4OSorK756F3/w==,-37
301966,einXHVwJjVdtb13FfVDbKg==,-36
360458,mw87rZC3q6OgJaW6ibEk5g==,-71
418091,v4uubF5e+ZaWzKiz0D1UpA==,-62


In [ ]:
## A partir de tener el top10, evaluar si esos atributos nos sirven para expandir el dataset
## Pensar en hacer un score con la completitud de la descripcion de cada propiedad -> property_desc_score

In [ ]:
Como es claro, la gente que publica la venta de una propiedad va a tratar de expresar la mejor publicacion y descripcion posible. Es por eso que tenemos una tasa altisima de positividad. No buscamos hacer un analisis real de a

In [25]:
def perfil_sentimientos(df):
    print(f"Positivos: {round(100 * np.mean(df.sentimiento > 0), 2)}")
    print(f"Neutros  : {round(100 * np.mean(df.sentimiento == 0), 2)}")
    print(f"Negativos: {round(100 * np.mean(df.sentimiento < 0), 2)}")
    print(" ")

perfil_sentimientos(descripcion_sentimientos)

Positivos: 89.67
Neutros  : 2.55
Negativos: 7.78
 


#### Tecnica Regex

Revisamos las siguientes paginas para entender cuales son los ammenities mas buscados en CABA, y en Argentina en general.

https://www.iprofesional.com/negocios/371702-cuales-son-los-amenities-mas-exoticos-de-edificios-en-argentina

https://www.baenegocios.com/sociedad/Ranking-de-amenities-los-servicios-que-mas-pesan-al-comprar-una-propiedad-20220119-0068.html

https://www.forbesargentina.com/negocios/amenities-servicios-mas-demandados-argentinos-comprar-una-propiedad-n11901

Dichos ammenities parecen hacer que la propiedad cotice entre un 15% y un 20% más que el precio de venta.

Sacando un promedio y haciendo un top-5 ranking, podemos notar que los mas relevantes son:

- Garage/Estacionamiento
- Pileta
- Jardin/Espacio al aire libre
- Parrilla
- SUM (Gimnasio/Spa/Sauna)

Al buscar estos datos, podriamos tratar de entender si el precio resulta mayor, contra una propiedad de similares caracteristicas pero sin estos ammenities y a partir de eso, entender que % varía del precio de venta original.

In [26]:
garage_sin_espacio = re.compile(r"[^\s]+garage|garaje|estacionamiento|parking")
garage_con_espacio = re.compile(r"[^\s]+ garage|garaje|estacionamiento|parking")

In [27]:
frecuencia_garage = {'garage': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(garage_sin_espacio, str(desc).lower())) > 0):
        frecuencia_garage['garage'] += 1
    if (len(re.findall(garage_con_espacio, str(desc).lower())) > 0):
        frecuencia_garage['garage'] += 1
print(frecuencia_garage)

KeyboardInterrupt: 

In [28]:
pileta_sin_espacio = re.compile(r"[^\s]+pileta")
pileta_con_espacio = re.compile(r"[^\s]+ pileta")

In [29]:
frecuencia_pileta= {'pileta': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(pileta_sin_espacio, str(desc).lower())) > 0):
        frecuencia_pileta['pileta'] += 1
    if (len(re.findall(pileta_con_espacio, str(desc).lower())) > 0):
        frecuencia_pileta['pileta'] += 1
print(frecuencia_pileta)

In [30]:
jardin_sin_espacio = re.compile(r"[^\s]+jardin|espacio verde")
jardin_con_espacio = re.compile(r"[^\s]+ jardin|espacio verde")

In [31]:
frecuencia_jardin = {'jardin': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(jardin_sin_espacio, str(desc).lower())) > 0):
        frecuencia_jardin['jardin'] += 1
    if (len(re.findall(jardin_con_espacio, str(desc).lower())) > 0):
        frecuencia_jardin['jardin'] += 1
print(frecuencia_jardin)

In [ ]:
parrilla_sin_espacio = re.compile(r"[^\s]+parrilla|bbq")
parrilla_con_espacio = re.compile(r"[^\s]+ parrilla|bbq")

In [ ]:
frecuencia_parrilla = {'parrilla': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(parrilla_sin_espacio, str(desc).lower())) > 0):
        frecuencia_parrilla['parrilla'] += 1
    if (len(re.findall(parrilla_con_espacio, str(desc).lower())) > 0):
        frecuencia_parrilla['parrilla'] += 1
print(frecuencia_parrilla)

In [ ]:
zoom_sin_espacio = re.compile(r"[^\s]+zoom|sum|gimansio|spa")
zoom_con_espacio = re.compile(r"[^\s]+ zoom|sum|gimansio|spa")

In [ ]:
frecuencia_zoom = {'sum': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(zoom_sin_espacio, str(desc).lower())) > 0):
        frecuencia_zoom['sum'] += 1
    if (len(re.findall(zoom_con_espacio, str(desc).lower())) > 0):
        frecuencia_zoom['sum'] += 1
print(frecuencia_zoom)

Notamos que muchas obervaciones tenian el dato de si contaban o no con balcon asi que decidimos agregarlo al analisis para poder ver si encontramos una correlacion que nos de mas informacion con lo que respecta al balcon y a la superifice total versus la superificie cubierta y el precio de venta de cada vivienda.

In [ ]:
balcon_sin_espacio = re.compile(r"[^\s]+balcon")
balcon_con_espacio = re.compile(r"[^\s]+ balcon")

In [ ]:
frecuencia_balcon= {'balcon': 0}
for desc in descriptions_dataset.property_description:
    if (len(re.findall(balcon_sin_espacio, str(desc).lower())) > 0):
        frecuencia_balcon['balcon'] += 1
    if (len(re.findall(balcon_con_espacio, str(desc).lower())) > 0):
        frecuencia_balcon['balcon'] += 1
print(frecuencia_balcon)

Otro factor interesante que podriamos analizar es a que tipo de vivienda pertenece cada ammenity. Y si donde encontramos una ammenity en particular, encontramos consecuentemente otra. Por ejemplo, una casa con jardin y parrila y/o pileta.
De esta manera podriamos tratar de determinar el costo de cada ammenity o como afecta al precio.

#### Tecnica Open Information Extraction

#### Modelos

## Redes Neuronales

### Regresión

### Clasificación

#### Construcción del target

#### Modelo

## Ensambles de modelos

### Ensamble 1

### Ensamble 2

## Conclusiones